In [1]:
import requests
import json

# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy
from scipy import ndimage

from matplotlib import cm
import matplotlib.pyplot as plt

import plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML

import ipywidgets as widgets
from ipywidgets import *

import cv2
import datetime as dt
from datetime import datetime, timedelta
import time

import warnings
warnings.filterwarnings('ignore')

from storage import *

%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

In [2]:
def getHeatmap(device, start, end):
    serial = devices[device][0]
    
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, start.strftime("%Y-%m-%dT%H:00:00"), end.strftime("%Y-%m-%dT%H:00:00"))
    
    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
    
    
    img = cv2.imread(devices[device][1])
    heatmap = cv2.resize(heatarr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    return combined_image

In [3]:
def getWeightedVelocityImage(device, startdate, enddate):
    try:
        serial = devices[device][0]
    except KeyError:
        raise Exception('Invalid device name.')
        
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))

    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
        
    blurred = ndimage.gaussian_filter1d(heatarr, 2, axis=1)
    blurred = ndimage.gaussian_filter(blurred, 5)
    
    gx, gy = np.gradient(blurred)

    Y  = np.arange(480)[::-1]
    X = np.arange(640)
    
    color = np.sqrt(gx**2 + gy**2) 
    max_c = np.max(color)

    color = color / max_c
    
    filter1 = np.where(blurred > 0.5, color, np.nan) # keep high density
    filter2 = np.where(color < 0.2, filter1,np.nan) # keep low gradient
    final = (1 - filter2) ** 2 * blurred # highlighting high density and low gradient even more
    
    img = cv2.imread(devices[device][1])
    
    final *= (1/np.nanmax(final)) # normalize array
    heatmap = np.delete(np.uint8(cm.Blues(final ** 2)*255), 3, axis=2) # convert heatmap to cv2 img shape
    combined_image = np.where(heatmap != heatmap[0][0], heatmap, img) # combine the images

    return combined_image

In [4]:
def dwellTimeZoneTable(device, parameter, startdate, enddate):
    if device == 'Streetscape':
        zoneIds = list(StreetscapeZones.values())
        inverted = inv_StreetscapeZones
    elif device == 'Outside':
        zoneIds = list(OutsideZones.values())
        inverted = inv_OutsideZones
    else:
        zoneIds = list(RaincoatZones.values())
        inverted = inv_RaincoatZones
    
    return_table = pd.DataFrame(columns=['Zone', parameter])

    for zone in zoneIds:
        dwellTimeZoneQuery = """
        query {
          zoneDwellTimeDistribution(
            zoneIds:%s,
            startTime:"%s",
            endTime:"%s",
            objClasses:["pedestrian"]) {
            edges {
              node {
                %s
              }
            }
          }
        }
        """ % ('[' + str(zone) + ']', startdate.strftime("%Y-%m-%dT00:00:00"), enddate.strftime("%Y-%m-%dT23:59:59"), parameter)
        response = requests.post(url, json={'query': dwellTimeZoneQuery}, headers = {'Authorization':token}).json()
        response_value = response['data']['zoneDwellTimeDistribution']['edges'][0]['node'][parameter]
        row = pd.DataFrame([[inverted[zone], response_value]], columns=['Zone', parameter])
        return_table = return_table.append(row)
    
    return return_table #.reset_index(drop='True').to_html()

In [5]:
def eventsDwellTimesGraph(device, startdate, enddate, parameter):
    serial = devices[device][0]

    dwellTimesQueries = """
        query {
          feedDwellTimeDistribution(
            serialnos: ["%s"],
            startTime:"%s",
            endTime:"%s",
            objClasses:["pedestrian"],
            interval:"1h") {
            edges {
              node {
                %s
                time
              }
            }
          }
        }
        """ % (serial, startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"), parameter)

    response = requests.post(url, json={'query': dwellTimesQueries}, headers = {'Authorization':token}).json()

    df = pd.DataFrame(response['data']['feedDwellTimeDistribution']['edges'])
    df['time'] = df['node'].apply(lambda x: x['time'])
    df[parameter] = df['node'].apply(lambda x: x[parameter])
    df = df.fillna(0).drop(columns=['node'])

    return go.Scatter(x=df['time'], y=df[parameter], mode='lines')

In [6]:
def create_df(serialno, startdate, enddate):
    query = """
        query {
          feedCountMetrics(
            serialnos:["%s"],
            startTime:"%s",
            endTime:"%s",
            objClasses:["pedestrian"],
            timezone:"America/New_York",
            interval:"24h") {
            edges {
              node {
                serialno
                result
                objClass
                time
              }
            }
          }
        }
        """ % (serialno, startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))
    
    df = pd.DataFrame(columns = ['Date', 'Number of Pedestrians', "Day of Week"])
    peds = requests.post(url, json={'query': query}, headers = {'Authorization':token})

    p = peds.json()['data']['feedCountMetrics']['edges']  
    
    for node in p:
        date = datetime.strptime(node['node']['time'][:10], '%Y-%m-%d')
        df = df.append({'Date': date, 'Number of Pedestrians': node['node']['result'], "Day of Week" : date.strftime('%A')}, ignore_index=True)
    
    df.set_index('Date', inplace=True)
    df['Number of Pedestrians'] = df['Number of Pedestrians'].astype('int32')
    return df

def remove_outliers(df):
    testing_df = df.drop(df[df['Number of Pedestrians'] == 0].index)
    q1 = testing_df['Number of Pedestrians'].quantile(0.10)
    q3 = testing_df['Number of Pedestrians'].quantile(0.90)

    interquartile_range = q3 - q1

    df = df[~((df['Number of Pedestrians'] < (q1)) \
             |(df['Number of Pedestrians'] > (q3)))]
    return df

def create_hourly_df(serialno, startdate, enddate):
    query = """
        query {
          feedCountMetrics(
            serialnos:["%s"],
            startTime:"%s",
            endTime:"%s",
            objClasses:["pedestrian"],
            timezone:"America/New_York",
            interval:"1h") {
            edges {
              node {
                serialno
                result
                objClass
                time
              }
            }
          }
        }
        """ % (serialno, startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))
    
    hour_peds = requests.post(url, json={'query': query}, headers = {'Authorization':token})
        
    ph = hour_peds.json()['data']['feedCountMetrics']['edges']
    
    hourly = pd.DataFrame(columns = ['Date', 'Number of Pedestrians', "Day of Week", "Hour"])
    for node in ph:
        date = node['node']['time'][:10]
        hr = node['node']['time'][11:16]
        time = date +' '+ hr
        date = datetime.strptime(time, '%Y-%m-%d %H:%M')
        hourly = hourly.append({'Date': date, 'Number of Pedestrians': node['node']['result'], "Day of Week" : date.strftime('%A'), "Hour": hr}, ignore_index=True)
    
    hourly.set_index('Date', inplace=True)

    hourly['Number of Pedestrians'] = hourly['Number of Pedestrians'].astype('int32')
    
    return hourly

def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + timedelta(days_ahead)

def get_maintenance(hours, device, day, startdate, enddate):
    
    serial = devices[device][0]

    saved_dates = []
    hourly = create_hourly_df(serial, startdate, enddate)

        
    start_date = pd.Timestamp(startdate)
    end_date = pd.Timestamp(enddate)
    
    plot_df = hourly.loc[(hourly.index >= start_date) & (hourly.index <= end_date)].copy()
    
    dates = []
    count = 0
    for index, row in plot_df.iterrows():
        if row['Number of Pedestrians'] > 0 & row['Number of Pedestrians'] < hours:
            count += 1
        if row['Number of Pedestrians'] >= hours/2:
            count += row['Number of Pedestrians']
        if count >= hours:
            if day == "Any":
                d = index + timedelta(days=1)
            else:
                d = next_weekday(index, time.strptime(day, "%A").tm_wday)
            d = d.replace(hour=16, minute=0, second=0)
            saved_dates.append(d)
            count = 0
    dates = pd.Series(saved_dates) 
    dates = dates.rename("Maintenance Dates")
    dates.drop_duplicates(keep='first', inplace=True)
    dates.to_list()
    
    df = create_df(serial, start_date, end_date)
        
    save = dates.to_frame()
    
    save=save.reset_index(drop = True)
    save.to_csv('maintenance.txt', sep=' ', index=False)
    
    # plotting the weekday barplot
    d = remove_outliers(df)
    plot_df = d.loc[(d.index >= start_date) & (d.index <= end_date)].copy()
    
    order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    p = plot_df.groupby(['Day of Week'])['Number of Pedestrians'].mean().reindex(order)
    fig = px.bar(p, x = p.index, y='Number of Pedestrians')
    fig.update_layout(title_text='Mean Counts for Pedestrians by Weekday Without Outliers', xaxis_title="Week Day")
    fig.show()
    # plotting the hourly barplot
    plot_df = hourly.loc[(hourly.index >= start_date) & (hourly.index <= end_date)].copy()
    
    plot_df["Hour"] = plot_df["Hour"].astype(str)
    plot_df["Hour"] = plot_df["Hour"].str.slice(0, 2, 1) 
    hour_df = pd.DataFrame()
    hour_df['Number of Pedestrians'] = plot_df.groupby('Hour')['Number of Pedestrians'].mean()
    fig = px.bar(hour_df, x = hour_df.index, y='Number of Pedestrians')
    fig.update_layout(title_text='Hourly Mean of Pedestrians', xaxis_title="Daytime Hour")
    fig.show()
    
    plot_df = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    fig = go.Figure()

    # Add scatter trace for line
    fig.add_trace(go.Scatter(
        x=plot_df.index,
        y=plot_df['Number of Pedestrians'],
        mode="lines",
        name="Pedestrian Counts"
    ))
    shapes = list()
    for date in dates:
            shapes.append(
                dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=date,
                y0=0,
                x1= date + timedelta(hours=9),
                y1=1,
                fillcolor="Red",
                opacity=0.5,
                layer="above",
                line_width=0,
            ))   
    
    fig.update_layout(
        shapes = shapes,
        title=f'Maintenance schedule between {start_date.date()} and {end_date.date()}.',
        xaxis_title="Date",
        yaxis_title="Number of Pedestrians",
    )
    # plotting the large maintenance schedule
    iplot(fig)

In [7]:

  
def get_by_event(device, event, time, cumulative, parameter):
    start = events[event][0]
    if not cumulative:
        start = start + dt.timedelta(hours=time)
        end = start + dt.timedelta(hours=1)
    else:
        end = start + dt.timedelta(hours=time)
    
    get_by_date(device, start, end, parameter)    
        
def get_by_date(device, startdate, enddate, parameter):
    fig = make_subplots(
        rows=3, cols=2,
        specs=[[{"rowspan": 2}, {"rowspan": 2}],
               [None, None],
               [{}, {}]],
        subplot_titles=("Heatmap","Desire Line Heatmap", 
                        f"Hourly Dwell Time {parameter.title()}", f"Dwell Time {parameter.title()} By Zone"))

    fig.add_trace(go.Image(z=getHeatmap(device, startdate, enddate)), 1, 1)
    fig.add_trace(go.Image(z=getWeightedVelocityImage(device, startdate, enddate)), 1, 2)
    
    for ax in ['xaxis', 'yaxis', 'xaxis2', 'yaxis2']:
        fig['layout'][ax].update(visible=False)
    
    dwell_table = dwellTimeZoneTable(device, parameter, startdate, enddate)
    fig.add_trace(go.Bar(x=dwell_table['Zone'], y=dwell_table[parameter]), 3, 2)
    
    fig['layout']['yaxis3']['title'].update(text="Time (s)")
    fig['layout']['yaxis4']['title'].update(text="Time (s)")
    
    fig['layout']['xaxis4'].update(tickangle=-45)

    
    dwell_graph = eventsDwellTimesGraph(device, startdate, enddate, parameter)
    fig.add_trace(dwell_graph, 3, 1)

    fig.update_layout(height=600, width=1000, showlegend=False)

    f2 = go.FigureWidget(fig)
    iplot(f2)        

#define widgets
device=widgets.Dropdown(options=list(devices.keys()))
parameter = ['mean', 'count', 'pct100', 'pct75', 'pct50', 'pct25']

event=widgets.Dropdown(options=list(events.keys()))
timerange=(0, 24) 
cumulative=widgets.Checkbox()

startdate=widgets.DatePicker(value=pd.to_datetime('2019-01-01'))
enddate=widgets.DatePicker(value=pd.to_datetime('2020-01-01'))
days=widgets.Dropdown(options=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", 
                               "Saturday", "Sunday", "Any"], value = "Saturday")

hours = widgets.IntSlider(min=0,max=2000,step=50,value=500)

#create tabs
tab_nest = widgets.Tab()
titles = ['Landing Page', 'View by Event', 'View by Date Range', 
          'Plan Maintenance', 'Privacy Statement', 'Zone Reference']

for i, name in enumerate(titles):
    tab_nest.set_title(i, name)

#interact function in isolation
f = interactive(get_by_event, device=device, event=event, time=timerange, cumulative=cumulative, parameter=parameter);
g = interactive(get_by_date, device=device, startdate=startdate, enddate=enddate, parameter=parameter);
h = interactive(get_maintenance, hours=hours, device=device, day=days, startdate=startdate, enddate=enddate)

tab_nest.children = [widgets.HTML(landing_page_html), 
                     VBox(children = f.children), 
                     VBox(children = g.children), 
                     VBox(children = h.children), 
                     widgets.HTML(privacy_statement_html),
                     widgets.HTML(zone_reference_html)]
display(tab_nest)